<a href="https://colab.research.google.com/github/ShadmanRohan/fsl-rsvae/blob/main/Basic_Feat_Gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preliminaries**

In [1]:
!git clone https://github.com/ShadmanRohan/fsl-rsvae.git

Cloning into 'fsl-rsvae'...
remote: Enumerating objects: 442, done.
remote: Counting objects: 100% (442/442), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 442 (delta 219), reused 402 (delta 195), pack-reused 0
Receiving objects: 100% (442/442), 486.68 KiB | 1.02 MiB/s, done.
Resolving deltas: 100% (219/219), done.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from distutils.dir_util import copy_tree
copy_tree("/content/gdrive/MyDrive/PAMI/AWA1_AWA2_SUN/data/AWA1", "/content/fsl-rsvae/datasets/AWA1")

['/content/fsl-rsvae/datasets/AWA1/seen_test.mat',
 '/content/fsl-rsvae/datasets/AWA1/seen_attribute.mat',
 '/content/fsl-rsvae/datasets/AWA1/seen_test_label.mat',
 '/content/fsl-rsvae/datasets/AWA1/seen_train.mat',
 '/content/fsl-rsvae/datasets/AWA1/unseen_attribute.mat',
 '/content/fsl-rsvae/datasets/AWA1/seen_train_label.mat',
 '/content/fsl-rsvae/datasets/AWA1/unseen_test.mat',
 '/content/fsl-rsvae/datasets/AWA1/unseen_test_label.mat']

In [4]:
import torch
import argparse
import numpy as np
from torch.autograd import Variable
#from torchvision.datasets.folder import DatasetFolder
from torch.utils.data import Dataset, DataLoader
from torch.distributions import uniform, normal
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
import torch.optim
import json
import torch.utils.data.sampler
import os
import glob
import random
import time
import pdb
import yaml
#import datasets.feature_loader as feat_loader
from sklearn.manifold import TSNE
import h5py
from scipy.stats import multivariate_normal
import scipy

**Dataloader**

In [35]:
import scipy.io
#features
data_train = scipy.io.loadmat('/content/fsl-rsvae/datasets/AWA1/seen_train.mat')
data_train = data_train['feature']
#data_train = np.transpose(data_train)


# labels
label_train = scipy.io.loadmat('/content/fsl-rsvae/datasets/AWA1/seen_train_label.mat')
data_train_label = label_train['label'][0]


# attr
tmp = scipy.io.loadmat('/content/fsl-rsvae/datasets/AWA1/seen_attribute.mat')
attr = tmp['seen_attribute']
#attr = np.transpose(attr)

In [36]:
class FeatureDataset(Dataset):

    def __init__(self, features, labels, attr):
        self.labels = labels
        self.features = features
        self.attr = attr

    def __len__(self):
        return len(data_train)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

feature_dataset = FeatureDataset(data_train, data_train_label, attr)
feature_loader = torch.utils.data.DataLoader(feature_dataset, shuffle=True, pin_memory=True, drop_last=False, batch_size=128) 

**Model**

In [37]:
class FeatsVAE(nn.Module):
    def __init__(self, x_dim, latent_dim, bottle_neck):
        super(FeatsVAE, self).__init__()

        self.x_dim = x_dim
        self.latent_dim = latent_dim
        self.bn1 = nn.BatchNorm1d(x_dim)
        self.relu = nn.ReLU(inplace=True)
        self.z_dist = normal.Normal(0, 1)
        self.init_weights()

        self.encoder = nn.Sequential(
            nn.Linear(self.x_dim+self.latent_dim, 1096),
            nn.LeakyReLU(),
            nn.Linear(1096, bottle_neck),
            nn.LeakyReLU())
        
        self.linear_mu =  nn.Sequential(
            nn.Linear(bottle_neck, latent_dim),
            nn.ReLU())
        self.linear_logvar =  nn.Sequential(
            nn.Linear(bottle_neck, latent_dim),
            nn.ReLU())
        
        self.decoder = nn.Sequential(
            nn.Linear(2*latent_dim, 512),  # 170
            nn.LeakyReLU(),
            nn.Linear(512, 1024),
            nn.LeakyReLU(),
            nn.Linear(1024, x_dim),  #2048
            #nn.Sigmoid(),
            nn.LeakyReLU(),
        )
        



    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)  
        eps = torch.randn_like(std)
        # remove abnormal points
        return mu + eps*std

    def init_weights(self):
        for m in self.modules():
          if isinstance(m, nn.Linear):
              m.weight.data.normal_(0, 0.02)
              m.bias.data.normal_(0, 0.02)

    def forward(self, x, attr):
        #print(x.size())
        #print(attr.size())

        x = torch.cat((x, attr), dim=1).to(torch.float32)   # 2048+85
        #print(x.size())
        #print(self.x_dim+self.latent_dim)
        
        x = self.encoder(x)    # 2048+85 -> 8
        #print(x.size())
        mu = self.linear_mu(x) # 8 -> 85
        #print(x)
        logvar = self.linear_logvar(x)  # 8 -> 85
        #print(logvar)
        latent_feats = self.reparameterize(mu, logvar)  # 85 (sample)
        #Z = self.z_dist.sample(attr.shape).cuda() 
        concat_feats = torch.cat((latent_feats, attr), dim=1)  # 85+85 (predecoding concat)

        recon_feats = self.decoder(concat_feats)  #85*2 -> 8
        #recon_feats = self.relu(self.bn1(recon_feats))
        recon_feats = self.relu(recon_feats)
        return mu, logvar, recon_feats

feats_vae = FeatsVAE(x_dim=2048, latent_dim=85, bottle_neck=128) # latent dim = attribute dim

**Train**

In [38]:
def train_vae(feature_loader, feats_vae, attributes):
    optimizer = torch.optim.Adam(feats_vae.parameters(), lr=0.001)
    feats_vae.train()
    feats_vae.cuda()
    #for ep in range(1000):
    for ep in range(4000):
      loss_recon_all = 0
      loss_kl_all = 0
      for idx, (data, label) in enumerate(feature_loader):
        print("training loop...")
        data = data
        
        #weight = weight.cuda() / torch.sum(weight)
        attr = torch.from_numpy(attributes[label]).float().cuda()
        data = data.cuda()
        #print(attr.device)
        #print(data.device)
        mu, logvar, recon_feats = feats_vae(data, attr)
        recon_loss = ((recon_feats - data)**2).mean(1)
        recon_loss = torch.mean(recon_loss)
        #kl_loss = -0.5*torch.sum(1+logvar-logvar.exp()-mu.pow(2)) / data.shape[0]
        kl_loss = (1+logvar-logvar.exp()-mu.pow(2)).sum(1)
        kl_loss = -0.5*torch.mean(kl_loss)
        L_vae = recon_loss+kl_loss
        optimizer.zero_grad()
        L_vae.backward()   
        optimizer.step()
        loss_recon_all += recon_loss.item()
        loss_kl_all += kl_loss.item()
        break
      print('Ep: %d   Recon Loss: %f   KL Loss: %f'%(ep, loss_recon_all/(idx+1), loss_kl_all/(idx+1)))
      print(recon_feats.shape)
    return feats_vae
    #torch.save({'state': feats_vae.state_dict()}, 'feats_vae_mini.pth') 

feats_vae = train_vae(feature_loader, feats_vae, attr)

Streaming output truncated to the last 5000 lines.
Ep: 2333   Recon Loss: 0.352403   KL Loss: 0.000000
torch.Size([128, 2048])
training loop...
Ep: 2334   Recon Loss: 0.341370   KL Loss: 0.000000
torch.Size([128, 2048])
training loop...
Ep: 2335   Recon Loss: 0.372211   KL Loss: 0.000000
torch.Size([128, 2048])
training loop...
Ep: 2336   Recon Loss: 0.350767   KL Loss: 0.000000
torch.Size([128, 2048])
training loop...
Ep: 2337   Recon Loss: 0.362662   KL Loss: 0.000000
torch.Size([128, 2048])
training loop...
Ep: 2338   Recon Loss: 0.408811   KL Loss: 0.000000
torch.Size([128, 2048])
training loop...
Ep: 2339   Recon Loss: 0.389102   KL Loss: 0.000000
torch.Size([128, 2048])
training loop...
Ep: 2340   Recon Loss: 0.388982   KL Loss: 0.000000
torch.Size([128, 2048])
training loop...
Ep: 2341   Recon Loss: 0.361486   KL Loss: 0.000000
torch.Size([128, 2048])
training loop...
Ep: 2342   Recon Loss: 0.391342   KL Loss: 0.000000
torch.Size([128, 2048])
training loop...
Ep: 2343   Recon Lo

**Inference and Save: Unseen Set**

In [39]:
data_unseen_test = scipy.io.loadmat('/content/fsl-rsvae/datasets/AWA1/unseen_test.mat')
data_unseen_test = data_unseen_test['feature']
#data_train = np.transpose(data_train)

# labels
label_unseen_test = scipy.io.loadmat('/content/fsl-rsvae/datasets/AWA1/unseen_test_label.mat')
label_unseen_test = label_unseen_test['label'][0]

# attr
unseen_attr = scipy.io.loadmat('/content/fsl-rsvae/datasets/AWA1/unseen_attribute.mat')
unseen_attr = unseen_attr['unseen_attribute']

#/content/fsl-rsvae/datasets/AWA1/unseen_attribute.mat

In [40]:
feature_un_dataset = FeatureDataset(data_unseen_test, label_unseen_test, unseen_attr)
feature_un_loader = torch.utils.data.DataLoader(feature_dataset, shuffle=False, drop_last=False, batch_size=len(feature_un_dataset))  # len(feature_un_dataset)

In [41]:
def generate_feats_with_noise(feats_vae, unseen_attr, samples_per_class=1000):
    feats_vae.eval()
    #feats_vae.to()
    feats_vae.to(torch.device('cuda:0'))
    #z_dist = normal.Normal(0, 1)
    ground_truths = list(range(len(unseen_attr)))*samples_per_class
    uns_atr = torch.from_numpy(np.array(unseen_attr)[ground_truths]).float().cuda() #to(torch.float32)
    #attr = torch.from_numpy(attributes[label])
    #attr = attr.repeat(ind_count, 1)

    z_dist = normal.Normal(0, 1)
    Z = z_dist.sample((samples_per_class*len(unseen_attr), 2048)).cuda()

    mu, logvar, recon_feats = feats_vae(Z, uns_atr)
    return ground_truths, recon_feats

noise_gt, noise_feats = generate_feats_with_noise(feats_vae, attr)
print(noise_feats.size())
print(np.shape(noise_gt))

torch.Size([40000, 2048])
(40000,)


In [42]:
len(noise_feats)

40000

In [43]:
noise_feats = noise_feats.cpu().detach().numpy()
noise_gt = np.array(noise_gt)
print(len(noise_feats))
recon_noise = {"reconstructed_noise" : noise_feats, "labels" : noise_gt}
scipy.io.savemat("reconstructed_noise.mat", recon_noise)

40000


In [44]:
#feature_loader = torch.utils.data.DataLoader(feature_dataset, shuffle=True, pin_memory=True, drop_last=False, batch_size=len(feature_un_dataset))

def generate_feats_with_image(feats_vae, data, labels, attrib):
    feats_vae.eval()
    #feats_vae.to()
    feats_vae.to(torch.device('cuda:0'))
    ground_truths = labels
    #print(torch.from_numpy(np.array((attr)[label])).to(torch.float32)
    #uns_atr = torch.from_numpy(np.array(attr)[label]).to(torch.float32)
    attrib = torch.from_numpy(np.array((attrib)[labels])).to(torch.float32).cuda()
    #print(uns_atr.shape)
    #print(data.shape)
    #print(uns_atr.shape)
    data = torch.from_numpy(data).cuda()
    mu, logvar, recon_feats = feats_vae(data, attrib)
    #print(recon_feats.shape)
    #ground_truths = ground_truths.cpu().detach().numpy()
    recon_feats = recon_feats.cpu().detach().numpy()

    return ground_truths, recon_feats

image_gt, image_feats = generate_feats_with_image(feats_vae, data_train, data_train_label, attr)
print(image_feats.shape)
print(image_gt.shape)

(19832, 2048)
(19832,)


In [45]:
recon_image = {"reconstructed_image" : image_feats, "labels" : image_gt}
scipy.io.savemat("reconstructed_image.mat", recon_image)

In [55]:
all_gt = np.concatenate([data_train_label, noise_gt, image_gt])
all_feats = np.concatenate([data_train, noise_feats, image_feats])

In [56]:
recon_all = {"reconstructed_feats_all" : all_feats, "labels" : all_gt}
scipy.io.savemat("reconstructed_all.mat", recon_all)

In [52]:
print(len(list(recon_all['reconstructed_feats_all'])))
print(len(list(recon_image['reconstructed_image'])))
print(len(list(recon_noise['reconstructed_noise'])))


TypeError: ignored

In [49]:
!cp /content/reconstructed_noise.mat /content/gdrive/MyDrive/generated_data

In [50]:
!cp /content/reconstructed_image.mat /content/gdrive/MyDrive/generated_data

In [58]:
!cp /content/reconstructed_all.mat /content/gdrive/MyDrive/generated_data

In [54]:
recon_all['reconstructed_feats_all']

79664